In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

base = '/mnt/data2/mxdi/archive/hf-mirror/open_llama_3b'
base_model = AutoModelForCausalLM.from_pretrained(base,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base)
pth_path = '/mnt/data2/mxdi/archive/FastChat/checkpoints'

peft_model = PeftModel.from_pretrained(
    base_model,
    pth_path,
    torch_dtype=torch.float16,
)

model = peft_model.merge_and_unload()
save_path = '/mnt/data2/mxdi/archive/models/rank8openllama3b'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

[2023-12-20 13:22:37,037] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


('/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer_config.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/special_tokens_map.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer.model',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/added_tokens.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer.json')

: 

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = '/mnt/data2/mxdi/archive/hf-mirror/llama-7b'
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/mxd/miniconda3/envs/xeval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [3]:
torch.cuda.is_available() 

True

In [6]:
model.to('cpu')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=1, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=1, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dr

In [2]:
pth_path = '/mnt/data2/mxdi/archive/FastChat/checkpoints/lora_rank8_shuffle/checkpoint-100'
peft_model = PeftModel.from_pretrained(
    model,
    pth_path,
    torch_dtype=torch.float16,
)

只能说雀食很奇怪

* 加载model不用to是不会到device上的

* 但是cpu的model再使用peft.from_pretrained的时候是会把部分加载到device上的

* 但是device上的model再使用peft的时候，device上的存储占用是不会变化的

In [1]:
import transformers

model_name_or_path = '/mnt/data2/mxdi/archive/models/Llama-2-7b-hf'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import torch

device = torch.device("cuda:0" )
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [7]:
next(model.parameters())

Parameter containing:
tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
          2.5787e-03, -3.9368e-03],
        [ 1.0986e-02,  9.8877e-03, -5.0964e-03,  ...,  2.5177e-03,
          7.7057e-04, -5.0049e-03],
        ...,
        [-1.3977e-02, -2.7313e-03, -1.9897e-02,  ..., -1.0437e-02,
          9.5825e-03, -1.8005e-03],
        [-1.0742e-02,  9.3384e-03,  1.2939e-02,  ..., -3.3203e-02,
         -1.6357e-02,  3.3875e-03],
        [-8.3008e-03, -4.0588e-03, -1.1063e-03,  ...,  3.4790e-03,
         -1.2939e-02,  3.1948e-05]], device='cuda:0', requires_grad=True)

In [9]:
model.device.index

0